## Import all the libraries

In [1]:
import numpy as np
import dask
import xarray as xr
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import glob as glob
import time
from dask.diagnostics import ProgressBar
from datetime import date
import json
import os
import warnings
import seawater
import re

warnings.filterwarnings('ignore')


## Parameters

In [2]:
ymin=2010;mmin=1;dmin=1
ymax=2010;mmax=4;dmax=30
# depth of the desired comparison profile in m
depthmin=500
# radius of the circle around the profile location in which we take the modeled profiles, in 째  
radius_max=0.25
# period of time around the profile sampling date in which we take the modeled profiles, in days
period=5
# minimum amount of model profiles to be considered to make a significant statistical comparison, for instance in a 1째 square and 30-days window we have 2.6 millions modeled profiles, in a 0.5째x10 days 216 000
number_of_model_profiles=100000


In [3]:
datemin=datetime.date(ymin,mmin,dmin)
datemax=datetime.date(ymax,mmax,dmax)
jsonfile='txt/MEDWEST60-BLBT02_'+str(datemin)+'-'+str(datemax)+'_'+str(depthmin)+'m_'+str(radius_max)+'x'+str(period)+'d_'+str(number_of_model_profiles)+'.json'

## Read the jsonfile

In [4]:
sourcefile=open(jsonfile,'rU')
infos=json.load(sourcefile)
nb_profilesEN4=len(infos)


In [13]:
vert_standart=np.array([0,2,4,6,8,10,12,14,16,18,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100,
                   110,120,130,140,150,160,170,180,190,200,220,240,260,280,300,320,340,360,380,400,
                   420,440,460,480,500,520,540,560,580,600,620,640,660,680,700,720,740,760,780,800,
                   820,840,860,880,900,920,940,960,980,1000,1050,1100,1150,1200,1250,1300,1350,1400,
                   1450,1500,1550,1600,1650,1700,1750,1800,1850,1900,1950,2000])
indz=np.where(vert_standart>depthmin)
vert_standart_red=vert_standart[0:np.min(indz[0])]

all_temp_mean_model=np.zeros((nb_profilesEN4,len(vert_standart_red)))
all_salt_mean_model=np.zeros((nb_profilesEN4,len(vert_standart_red)))
all_temp_en4=np.zeros((nb_profilesEN4,len(vert_standart_red)))
all_salt_en4=np.zeros((nb_profilesEN4,len(vert_standart_red)))
all_temp_percent10_model=np.zeros((nb_profilesEN4,len(vert_standart_red)))
all_salt_percent10_model=np.zeros((nb_profilesEN4,len(vert_standart_red)))
all_temp_percent90_model=np.zeros((nb_profilesEN4,len(vert_standart_red)))
all_salt_percent90_model=np.zeros((nb_profilesEN4,len(vert_standart_red)))


In [14]:
print(vert_standart_red)

[  0   2   4   6   8  10  12  14  16  18  20  25  30  35  40  45  50  55
  60  65  70  75  80  85  90  95 100 110 120 130 140 150 160 170 180 190
 200 220 240 260 280 300 320 340 360 380 400 420 440 460 480 500]


In [15]:
print(nb_profilesEN4)

131


In [16]:
for prof in np.arange(nb_profilesEN4):
    list_profiles = infos.keys()
    reference =  str(list(list_profiles)[prof])
    print('Processing profile ', reference)
    # Open the netcdf file
    match=re.search(r'([\w.-]+).nc([\w.-]+)', reference)
    debut_ref=match.group(1)
    fin_ref=match.group(2)
    diren4="/gpfswork/rech/egi/rote001/EN4/"
    dirname=diren4+'profiles_files/MEDWEST60-BLBT02/'
    outname=dirname+str(debut_ref)+str(fin_ref)+'_MEDWEST60-BLBT02_TS.nc'
    dsprof=xr.open_dataset(outname)

    depth_en4=dsprof['depth_en4']
    temp_en4=dsprof['temp_profileEN4']
    salt_en4=dsprof['salt_profileEN4']
    mean_temp_model=dsprof['mean_temp_model']
    mean_salt_model=dsprof['mean_salt_model']
    percent10_temp_model=dsprof['percent10_temp_model']
    percent10_salt_model=dsprof['percent10_salt_model']
    percent90_temp_model=dsprof['percent90_temp_model']
    percent90_salt_model=dsprof['percent90_salt_model']
    obs_lat=dsprof['latitude_profileEN4']
    obs_lon=dsprof['longitude_profileEN4']
    obs_time=dsprof['time_profileEN4']
    
    # Project the profile to standart vertical levels
    temp_en4_standart=np.interp(vert_standart_red,depth_en4,temp_en4)
    salt_en4_standart=np.interp(vert_standart_red,depth_en4,salt_en4)
    mean_temp_model_standart=np.interp(vert_standart_red,depth_en4,mean_temp_model)
    mean_salt_model_standart=np.interp(vert_standart_red,depth_en4,mean_salt_model)
    percent10_temp_model_standart=np.interp(vert_standart_red,depth_en4,percent10_temp_model)
    percent10_salt_model_standart=np.interp(vert_standart_red,depth_en4,percent10_salt_model)
    percent90_temp_model_standart=np.interp(vert_standart_red,depth_en4,percent90_temp_model)
    percent90_salt_model_standart=np.interp(vert_standart_red,depth_en4,percent90_salt_model)

    #Fill the all profiles
    all_temp_mean_model[prof,:]=mean_temp_model_standart[:]
    all_salt_mean_model[prof,:]=mean_salt_model_standart[:]
    all_temp_en4[prof,:]=temp_en4_standart[:]
    all_salt_en4[prof,:]=salt_en4_standart[:]
    all_temp_percent10_model[prof,:]=percent10_temp_model_standart[:]
    all_salt_percent10_model[prof,:]=percent10_salt_model_standart[:]
    all_temp_percent90_model[prof,:]=percent90_temp_model_standart[:]
    all_salt_percent90_model[prof,:]=percent90_salt_model_standart[:]


Processing profile  Profile_20100106_prof.nc_71
Processing profile  Profile_20100106_prof.nc_76
Processing profile  Profile_20100108_prof.nc_90
Processing profile  Profile_20100108_prof.nc_92
Processing profile  Profile_20100109_prof.nc_0
Processing profile  Profile_20100109_prof.nc_62
Processing profile  Profile_20100110_prof.nc_0
Processing profile  Profile_20100110_prof.nc_81
Processing profile  Profile_20100113_prof.nc_69
Processing profile  Profile_20100114_prof.nc_3
Processing profile  Profile_20100114_prof.nc_44
Processing profile  Profile_20100115_prof.nc_0
Processing profile  Profile_20100115_prof.nc_58
Processing profile  Profile_20100116_prof.nc_79
Processing profile  Profile_20100118_prof.nc_85
Processing profile  Profile_20100118_prof.nc_88
Processing profile  Profile_20100119_prof.nc_1
Processing profile  Profile_20100119_prof.nc_58
Processing profile  Profile_20100120_prof.nc_0
Processing profile  Profile_20100120_prof.nc_91
Processing profile  Profile_20100123_prof.nc_7

In [17]:
all_temp_en4.shape

(131, 52)

In [36]:
all_temp_percent10_model[9,:]

array([15.40994653, 15.40994653, 15.40994653, 15.40994653, 15.40994653,
       15.40994653, 15.40994653, 15.40994653, 15.40964265, 15.40909708,
       15.40855151, 15.4071876 , 15.40542499, 15.40358626, 15.40131663,
       15.39891528, 15.3957225 , 15.39252972, 15.38913155, 15.38570702,
       15.38248363, 15.37928212, 15.37559207, 15.37185825, 15.35766221,
       15.34272282, 15.25297183, 14.97562697, 14.6354975 , 14.36226243,
       14.15040508, 13.98139847, 13.84070873, 13.72513594, 13.63109087,
       13.55676548, 13.49934812, 13.41986998, 13.37107197, 13.33898476,
       13.31659977, 13.29750329, 13.27814644, 13.25594368, 13.23229806,
        7.14170343,         nan,         nan,         nan,         nan,
               nan,         nan])

In [34]:
ind=np.where(np.isnan(all_temp_percent10_model[:,-1])==True)
print(np.min(ind[0]))

9


In [35]:
print(ind[0])

[  9  39  43  44  50  55  63  66  67 111 113 119 122 127]


In [27]:
all_temp_en4[52,:]

array([14.        , 14.        , 14.00002697, 14.00179492, 14.00356287,
       14.00474876, 14.003405  , 14.00206124, 14.00347932, 14.00502961,
       14.00632076, 14.00845594, 14.01049371, 14.01221589, 14.01245013,
       14.01160572, 14.01542389, 14.02122552, 14.0210886 , 14.03045195,
       14.02400017, 14.02980122, 14.06701492, 14.09905491, 14.11195364,
       14.15648448, 14.17350874, 14.14769499, 13.98141189, 14.01663426,
       13.95780943, 13.88208992, 13.68283078, 13.37508119, 13.18517005,
       13.1572153 , 13.13084645, 13.07562469, 13.06514817, 13.43480736,
               nan,         nan,         nan, 13.64941168, 13.63399982,
       13.62041968, 13.56799881, 13.52129767, 13.508136  , 13.50362349,
       13.48700047, 13.46197221])

In [37]:
    prof=9
    list_profiles = infos.keys()
    reference =  str(list(list_profiles)[prof])
    print('Processing profile ', reference)
    # Open the netcdf file
    match=re.search(r'([\w.-]+).nc([\w.-]+)', reference)
    debut_ref=match.group(1)
    fin_ref=match.group(2)
    diren4="/gpfswork/rech/egi/rote001/EN4/"
    dirname=diren4+'profiles_files/MEDWEST60-BLBT02/'
    outname=dirname+str(debut_ref)+str(fin_ref)+'_MEDWEST60-BLBT02_TS.nc'


Processing profile  Profile_20100114_prof.nc_3


In [38]:
print(outname)

/gpfswork/rech/egi/rote001/EN4/profiles_files/MEDWEST60-BLBT02/Profile_20100114_prof_3_MEDWEST60-BLBT02_TS.nc


In [39]:
mean_all_temp_mean_model=np.nanmean(all_temp_mean_model,axis=0)
mean_all_salt_mean_model=np.nanmean(all_salt_mean_model,axis=0)
mean_all_temp_en4=np.nanmean(all_temp_en4,axis=0)
mean_all_salt_en4=np.nanmean(all_salt_en4,axis=0)
mean_all_temp_percent10_model=np.nanmean(all_temp_percent10_model,axis=0)
mean_all_salt_percent10_model=np.nanmean(all_salt_percent10_model,axis=0)
mean_all_temp_percent90_model=np.nanmean(all_temp_percent90_model,axis=0)
mean_all_salt_percent90_model=np.nanmean(all_salt_percent90_model,axis=0)


In [32]:
print(mean_all_temp_percent10_model)

[14.74629955 14.74629955 14.74629948 14.74572508 14.74498768 14.74328429
 14.74139835 14.73929474 14.73664391 14.73294347 14.72873847 14.71739518
 14.70408882 14.6901359  14.67559728 14.66032581 14.6445837  14.62860852
 14.6123679  14.59597528 14.57927985 14.56260089 14.54336399 14.5227015
 14.49727594 14.47038948 14.4384851  14.36862551 14.2865529  14.19313996
 14.09074812 13.98754401 13.74673781         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan]


In [33]:
print(all_temp_percent10_model[:,-1])

[13.18400127 13.21204943 13.16827397 13.20181206 13.12718717 13.14543171
 13.17901424 13.26234886 13.19469551         nan 13.16492443 13.17561307
 13.22442552 13.20203139 13.17532232 13.18488483 13.12105927 13.165121
 13.16860975 13.22827726 13.19665301 13.18122547 13.18110383 13.22323075
 13.22251657 13.16962333 13.20109903 13.12410938 13.16722866 13.19417246
 13.18461764 13.20864189 13.16263587 13.18061926 13.20528092 13.21886619
 13.24598561 13.18756294 13.11547925         nan 13.17060259 13.24292496
 13.21103648         nan         nan 13.20980292 13.21827165 13.16667456
 13.19058319 13.19225245         nan 13.17361813 13.20184508 13.11921087
 13.23163365         nan 13.15678664 13.21018935 13.17945567 13.21732591
 13.20165909 13.16822869 13.11783622         nan 13.19037114 13.20878141
         nan         nan 13.17213655 13.17103356 13.19554409 13.21906096
 13.23079503 13.14050232 13.15691296 13.15694253 13.16568091 13.11750934
 13.16101455 13.16108109 13.2118777  13.19892276 13.1

In [40]:
min_temp=np.nanmin([np.nanmin(mean_all_temp_mean_model),
                 np.nanmin(mean_all_temp_en4),
                 np.nanmin(mean_all_temp_percent10_model),
                 np.nanmin(mean_all_temp_percent90_model)])
max_temp=np.nanmax([np.nanmax(mean_all_temp_mean_model),
                 np.nanmax(mean_all_temp_en4),
                 np.nanmax(mean_all_temp_percent10_model),
                 np.nanmax(mean_all_temp_percent90_model)])
min_salt=np.nanmin([np.nanmin(mean_all_salt_mean_model),
                 np.nanmin(mean_all_salt_en4),
                 np.nanmin(mean_all_salt_percent10_model),
                 np.nanmin(mean_all_salt_percent90_model)])
max_salt=np.nanmax([np.nanmax(mean_all_salt_mean_model),
                 np.nanmax(mean_all_salt_en4),
                 np.nanmax(mean_all_salt_percent10_model),
                 np.nanmax(mean_all_salt_percent90_model)])
print(min_temp,max_temp,min_salt,max_salt)



12.87333052897434 15.222330959298416 37.56800786100736 38.53979215177431


In [9]:
print(np.nanmin(mean_all_salt_mean_model),
                 np.nanmin(mean_all_salt_en4),
                 np.nanmin(mean_all_salt_percent10_model),
                 np.nanmin(mean_all_salt_percent90_model))

37.628761018252426 37.90362621627691 37.69102173965656 37.89310071261223


In [41]:
fig, axs = plt.subplots(1,3, figsize=(15, 6))
axs = axs.ravel()
title = 'Comparison of Temperature and Salinity Profiles with '+str(nb_profilesEN4)+' profiles of EN4 database'
plt.suptitle(title,size = 15,y=1.05)
axs[0].plot(mean_all_temp_en4,vert_standart_red,'k.-', label='temp en4')
axs[0].plot(mean_all_temp_mean_model,vert_standart_red,'b.-', label='temp model-mean')
axs[0].set_ylabel('Depth [m]', size=14)
axs[0].set_xlim(left=min_temp-(max_temp-min_temp)/10,right=max_temp+(max_temp-min_temp)/10)
axs[0].set_ylim(depthmin, 0)
axs[0].grid(True, which='both')
axs[0].xaxis.tick_top()
axs[0].xaxis.set_label_position('top') 
axs[0].plot(mean_all_temp_percent10_model,vert_standart_red,'b-', label='temp model-percent10')
axs[0].plot(mean_all_temp_percent90_model,vert_standart_red,'b-', label='temp model-percent90')
axs[0].fill_betweenx(vert_standart_red, mean_all_temp_percent10_model, x2=mean_all_temp_percent90_model, alpha=0.2, facecolor='b')
#        axs[0].set_title('Temperature profiles')
axs[0].legend()
        
axs[1].plot(mean_all_salt_en4,vert_standart_red,'k.-', label='salt en4')
axs[1].plot(mean_all_salt_mean_model,vert_standart_red,'b.-', label='salt model-mean')
axs[1].set_ylabel('Depth [m]', size=14)
axs[1].set_xlim(left=min_salt-(max_salt-min_salt)/10,right=max_salt+(max_salt-min_salt)/10)
axs[1].set_ylim(depthmin, 0)
axs[1].grid(True, which='both')
axs[1].xaxis.tick_top()
axs[1].xaxis.set_label_position('top') 
axs[1].plot(mean_all_salt_percent10_model,vert_standart_red,'b-', label='salt model-percent10')
axs[1].plot(mean_all_salt_percent90_model,vert_standart_red,'b-', label='salt model-percent90')
axs[1].fill_betweenx(vert_standart_red, mean_all_salt_percent10_model, x2=mean_all_salt_percent90_model, alpha=0.2, facecolor='b')
#        axs[1].set_title('Salinity profiles')
axs[1].legend()

textstr = '\n'.join((
            ' simulation = MEDWEST60-BLBT02',
            ' dates = '+str(datemin)+' '+str(datemax),
            ' radius max = '+str(radius_max)+'째',
            ' period = '+str(period)+'d',
            ' depth min = '+str(depthmin)+'m',
            ' nb_profiles = '+str(number_of_model_profiles)))        
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
axs[2].text(0.05, 0.95, textstr, transform=axs[2].transAxes, fontsize=14,verticalalignment='top', bbox=props)
axs[2].axis('off')
fig.tight_layout()
plt.savefig('plots/profiles-TS-MEDWEST60-BLBT02_'+str(datemin)+'-'+str(datemax)+'_'+str(depthmin)+'m_'+str(radius_max)+'x'+str(period)+'d_'+str(number_of_model_profiles)+'.png',bbox_inches='tight')
